In [1]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher, get_close_matches
from tqdm import tqdm
import re

In [2]:
df_manufacturer = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/unmatched_manufacturer_stockist_transactions.csv')
df_manufacturer = df_manufacturer.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
df_manufacturer.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score
0,shalina healthcare ltd,viva healthcare ltd.,0.79
1,pi industries ltd,zaam industries ltd.,0.79
2,kitale industries,kitale industries limited,0.79
3,avivet agro solution ltd,agrosolutions ltd.,0.79
4,local manufacturer,"""""manufacturer",0.79


In [3]:
df_non_dup = df_manufacturer.drop_duplicates(subset='manufacturer_name', keep='first')
df_non_dup['word_count'] = df_manufacturer['manufacturer_name'].apply(lambda x: len(x.split()) if isinstance(x, str) else 1)
df_non_dup['manufacturer_name'] = df_non_dup['manufacturer_name'].astype('str')

manufacturer_list = []

for index, row in df_non_dup.iterrows():
    word_count = row['word_count']
    manufacturer_name = row['manufacturer_name']
    
    if word_count in [1,2]:
        manufacturer_slice = manufacturer_name.strip().split()[:1]
        manufacturer_list.append(' '.join(manufacturer_slice))
    elif word_count in [3,4,5]:
        manufacturer_slice = manufacturer_name.strip().split()[:2]
        manufacturer_list.append(' '.join(manufacturer_slice))
    else:
        manufacturer_slice = manufacturer_name.strip().split()[:3]
        manufacturer_list.append(' '.join(manufacturer_slice))
           
df_non_dup['manufacturer_slice'] = manufacturer_list        
df_non_dup = df_non_dup.reset_index(drop=True)
df_non_dup.head()

/tmp/ipykernel_11104/1627982089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_dup['word_count'] = df_manufacturer['manufacturer_name'].apply(lambda x: len(x.split()) if isinstance(x, str) else 1)
/tmp/ipykernel_11104/1627982089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_dup['manufacturer_name'] = df_non_dup['manufacturer_name'].astype('str')
/tmp/ipykernel_11104/1627982089.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice
0,shalina healthcare ltd,viva healthcare ltd.,0.79,3,shalina healthcare
1,pi industries ltd,zaam industries ltd.,0.79,3,pi industries
2,kitale industries,kitale industries limited,0.79,2,kitale
3,avivet agro solution ltd,agrosolutions ltd.,0.79,4,avivet agro
4,local manufacturer,"""""manufacturer",0.79,2,local


In [4]:
df_non_dup.shape

(30392, 5)

In [5]:
df_manufacturer_slice = df_non_dup['manufacturer_slice'].drop_duplicates(keep='first').to_frame()
df_manufacturer_slice = df_manufacturer_slice.reset_index(drop=True)
df_manufacturer_slice

,manufacturer_slice
0,shalina healthcare
1,pi industries
2,kitale
3,avivet agro
4,local
...,...
20170,0
20171,#
20172,@
20173,*


In [ ]:
# matched = {'key': ['value']}
# def compare(i):
#     compare = {}
#     for key, value in matched.items():
#       if i in value:
#         compare.update({i: value})
#       else:
#         compare.update({i: get_close_matches(i, df_manufacturer_slice['manufacturer_slice'].to_list(), 5, 0.8 )})
#     matched.update(compare.items())
#     # manufacturer_slice = list(compare.keys())
#     # match = []len(matched)
#     # for key, items in compare.items():
#     #   match.append(items)
#     # return pd.Series([manufacturer_slice, match],index=['kemsa_prod', 'match'])
#     return matched
      
# for i in tqdm(range(10), desc="Processing", ncols=80, bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}"):
#   # cleaned_manufacturers_df = pd.DataFrame()
#   # cleaned_manufacturers_df[['manufacturer_slice', 'match']] = df_manufacturer_slice['manufacturer_slice'].apply(lambda x: compare(x))
#   # cleaned_manufacturers_df = cleaned_manufacturers_df.applymap(lambda x: x[0] if x else '')
#   # cleaned_manufacturers_df.head()
#   matched_df = df_manufacturer_slice['manufacturer_slice'].apply(lambda x: compare(x))

# # x = compare('pioneer  feeds')
# # cleaned_manufacturers_df = pd.DataFrame()
# # cleaned_manufacturers_df[['manufacturer_slice', 'match']] = x
# # cleaned_manufacturers_df['match'][0]
# x

In [6]:
matched = []
def compare(i):
    compare = {}
    if i in matched:
        compare.update({i: ''})
    else:
        compare.update({i: get_close_matches(i, df_manufacturer_slice['manufacturer_slice'].to_list(), 20, 0.85)})
    # matched.extend(compare.values())
    matched.extend([item for sublist in compare.values() for item in sublist])
    manufacturer_slice = list(compare.keys())
    match = []
    for key, items in compare.items():
      match.append(items)
    return pd.Series([manufacturer_slice, match],index=['manufacturer_slice', 'match'])

      
cleaned_manufacturers_df = pd.DataFrame()
cleaned_manufacturers_df[['manufacturer_slice', 'match']] = df_manufacturer_slice['manufacturer_slice'].apply(lambda x: compare(x))
cleaned_manufacturers_df = cleaned_manufacturers_df.applymap(lambda x: x[0] if x else '')
cleaned_manufacturers_df.head()

,manufacturer_slice,match
0,shalina healthcare,"[shalina healthcare, shelina healthcare, shali..."
1,pi industries,"[pi industries, pi industies, pt industries, p..."
2,kitale,[kitale]
3,avivet agro,"[avivet agro, avvivet agro, avivet ago, avivet..."
4,local,"[local, lockal, localy, locals, localp, locall..."


In [7]:
len(set(matched))

20175

In [8]:
dnd_cleaned_manufacturers_df = cleaned_manufacturers_df.copy()
cleaned_manufacturers_df['manufacturer_slice'] = cleaned_manufacturers_df['manufacturer_slice'].astype('str')

In [9]:
filtered_df = cleaned_manufacturers_df[cleaned_manufacturers_df['match'].apply(lambda x: len(x) == 0)]
filtered_df

,manufacturer_slice,match
17,pt industries,
25,western seed,
28,inter-agro,
42,arom chemicals,
45,delta chemical,
...,...,...
20141,100,
20143,350,
20146,1150,
20150,triplel,


In [28]:
# cleaned_manufacturers_df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/model/cleaned_manufacturers.csv')
# cleaned_manufacturers_df

In [10]:
for i, row in cleaned_manufacturers_df.iterrows():
    string = row['manufacturer_slice']
    lst = row['match']
    
    # Check if the string exists in any list of previous rows
    if not lst:
        for prev_i in range(i):
            prev_lst = cleaned_manufacturers_df.at[prev_i, 'match']
            if string in prev_lst:
                cleaned_manufacturers_df.at[i, 'match'] = prev_lst
                break  # Stop searching after finding the first match

# Reset the index of the DataFrame
# cleaned_manufacturers_df_subset = cleaned_manufacturers_df_subset.reset_index(drop=True)

pd.set_option('display.max_colwidth', None)
cleaned_manufacturers_df.head()

,manufacturer_slice,match
0,shalina healthcare,"[shalina healthcare, shelina healthcare, shalina health]"
1,pi industries,"[pi industries, pi industies, pt industries, p1 industries, ip industries]"
2,kitale,[kitale]
3,avivet agro,"[avivet agro, avvivet agro, avivet ago, avivet. agro.]"
4,local,"[local, lockal, localy, locals, localp, locall, local., local,, llocal, locl, loca, loal, lcal]"


In [11]:
dnd_df_non_dup = df_non_dup.copy()

In [12]:
df_non_dup = pd.merge(df_non_dup, cleaned_manufacturers_df, how='left', on='manufacturer_slice')
df_non_dup.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice,match
0,shalina healthcare ltd,viva healthcare ltd.,0.79,3,shalina healthcare,"[shalina healthcare, shelina healthcare, shalina health]"
1,pi industries ltd,zaam industries ltd.,0.79,3,pi industries,"[pi industries, pi industies, pt industries, p1 industries, ip industries]"
2,kitale industries,kitale industries limited,0.79,2,kitale,[kitale]
3,avivet agro solution ltd,agrosolutions ltd.,0.79,4,avivet agro,"[avivet agro, avvivet agro, avivet ago, avivet. agro.]"
4,local manufacturer,"""""manufacturer",0.79,2,local,"[local, lockal, localy, locals, localp, locall, local., local,, llocal, locl, loca, loal, lcal]"


In [13]:
# Initialize an empty list to store the similar strings
similar_strings = []

# Iterate over the rows of the DataFrame
for i, row in df_non_dup.iterrows():
    string = row['manufacturer_name']
    value = row['match']
    
    # Check if any other row has a similar value in 'col2'
    similar_rows = df_non_dup[df_non_dup['match'].apply(lambda x: x == value)]
    
    # Extract the strings from 'col1' in similar rows
    similar_strings.append(similar_rows['manufacturer_name'].tolist())

similar_strings[:5]

[['shalina healthcare ltd',
  'shalina healthcare kenya limited',
  'shalina healthcare kenya ltd',
  'shelina healthcare kenya limited',
  'shalina health kenya limited',
  'shalina health kenya'],
 ['pi industries ltd',
  'pt industries ltd',
  'p1 industries ltd',
  'ip industries ltd',
  'pi industies ltd'],
 ['kitale industries',
  'kitale indusries',
  'kitale industrues',
  'kitale indistries',
  'kitale industriez',
  'kitale induatries',
  'kitale millers',
  'kitale ltd',
  'kitale feeds',
  'kitale industry',
  'kitale',
  'kitale indust.',
  'kitale  ltd',
  'kitale best',
  'kitale agritech',
  'kitale agriteeh',
  'kitale agreetec'],
 ['avivet agro solution ltd',
  'avivet agro ltd',
  'avivet agro solutions lmd',
  'avivet agro sutions ltd',
  'avivet agro solutions limited',
  'avivet agro chemicals',
  'avivet.  agro.   solutions',
  'avivet agro solution limited',
  'avivet agro solutions',
  'avivet agro solution',
  'avivet agro soln ltd',
  'avivet ago solutions',


In [14]:
df_similar_strings = pd.DataFrame({'similar_strings': similar_strings})
df_similar_strings = df_similar_strings['similar_strings'].drop_duplicates(keep='first').to_frame().reset_index(drop=True)
df_similar_strings

,similar_strings
0,"[shalina healthcare ltd, shalina healthcare kenya limited, shalina healthcare kenya ltd, shelina healthcare kenya limited, shalina health kenya limited, shalina health kenya]"
1,"[pi industries ltd, pt industries ltd, p1 industries ltd, ip industries ltd, pi industies ltd]"
2,"[kitale industries, kitale indusries, kitale industrues, kitale indistries, kitale industriez, kitale induatries, kitale millers, kitale ltd, kitale feeds, kitale industry, kitale, kitale indust., kitale ltd, kitale best, kitale agritech, kitale agriteeh, kitale agreetec]"
3,"[avivet agro solution ltd, avivet agro ltd, avivet agro solutions lmd, avivet agro sutions ltd, avivet agro solutions limited, avivet agro chemicals, avivet. agro. solutions, avivet agro solution limited, avivet agro solutions, avivet agro solution, avivet agro soln ltd, avivet ago solutions, avvivet agro solution, avivet agro sokutions, avivet agro solution(, avivet agro sol]"
4,"[local manufacturer, local juakali, local manufacturers, local manufaturer, local manufacture, local suppliers, local seller, local industries, local ltd, local ltd., local dealers, locals, local carpenters, local any, local potter, localy made, local bomet, locall, local supplier, local indurtries, local market, loca, local providers, local nairobi, local fabricators, local products, local artisan, local welder, local nuteez, localy, lockal, local gas, lcal, loal, locl, local, llocal, local., local farmer, local plastic, local,, localp]"
...,...
14186,[0]
14187,[#]
14188,[@]
14189,[*]


In [15]:
df_unique_match = df_non_dup['match'].drop_duplicates(keep='first').to_frame().reset_index(drop=True)
df_unique_match = pd.concat([df_unique_match, df_similar_strings],axis = 1)
df_unique_match.head()

,match,similar_strings
0,"[shalina healthcare, shelina healthcare, shalina health]","[shalina healthcare ltd, shalina healthcare kenya limited, shalina healthcare kenya ltd, shelina healthcare kenya limited, shalina health kenya limited, shalina health kenya]"
1,"[pi industries, pi industies, pt industries, p1 industries, ip industries]","[pi industries ltd, pt industries ltd, p1 industries ltd, ip industries ltd, pi industies ltd]"
2,[kitale],"[kitale industries, kitale indusries, kitale industrues, kitale indistries, kitale industriez, kitale induatries, kitale millers, kitale ltd, kitale feeds, kitale industry, kitale, kitale indust., kitale ltd, kitale best, kitale agritech, kitale agriteeh, kitale agreetec]"
3,"[avivet agro, avvivet agro, avivet ago, avivet. agro.]","[avivet agro solution ltd, avivet agro ltd, avivet agro solutions lmd, avivet agro sutions ltd, avivet agro solutions limited, avivet agro chemicals, avivet. agro. solutions, avivet agro solution limited, avivet agro solutions, avivet agro solution, avivet agro soln ltd, avivet ago solutions, avvivet agro solution, avivet agro sokutions, avivet agro solution(, avivet agro sol]"
4,"[local, lockal, localy, locals, localp, locall, local., local,, llocal, locl, loca, loal, lcal]","[local manufacturer, local juakali, local manufacturers, local manufaturer, local manufacture, local suppliers, local seller, local industries, local ltd, local ltd., local dealers, locals, local carpenters, local any, local potter, localy made, local bomet, locall, local supplier, local indurtries, local market, loca, local providers, local nairobi, local fabricators, local products, local artisan, local welder, local nuteez, localy, lockal, local gas, lcal, loal, locl, local, llocal, local., local farmer, local plastic, local,, localp]"


In [16]:
dnd_df_unique_match = df_unique_match.copy()

In [17]:
df_unique_match = dnd_df_unique_match
df_unique_match['similar_strings'] = df_unique_match['similar_strings'].apply(lambda x: ' '.join(x))
df_unique_match

,match,similar_strings
0,"[shalina healthcare, shelina healthcare, shalina health]",shalina healthcare ltd shalina healthcare kenya limited shalina healthcare kenya ltd shelina healthcare kenya limited shalina health kenya limited shalina health kenya
1,"[pi industries, pi industies, pt industries, p1 industries, ip industries]",pi industries ltd pt industries ltd p1 industries ltd ip industries ltd pi industies ltd
2,[kitale],kitale industries kitale indusries kitale industrues kitale indistries kitale industriez kitale induatries kitale millers kitale ltd kitale feeds kitale industry kitale kitale indust. kitale ltd kitale best kitale agritech kitale agriteeh kitale agreetec
3,"[avivet agro, avvivet agro, avivet ago, avivet. agro.]",avivet agro solution ltd avivet agro ltd avivet agro solutions lmd avivet agro sutions ltd avivet agro solutions limited avivet agro chemicals avivet. agro. solutions avivet agro solution limited avivet agro solutions avivet agro solution avivet agro soln ltd avivet ago solutions avvivet agro solution avivet agro sokutions avivet agro solution( avivet agro sol
4,"[local, lockal, localy, locals, localp, locall, local., local,, llocal, locl, loca, loal, lcal]","local manufacturer local juakali local manufacturers local manufaturer local manufacture local suppliers local seller local industries local ltd local ltd. local dealers locals local carpenters local any local potter localy made local bomet locall local supplier local indurtries local market loca local providers local nairobi local fabricators local products local artisan local welder local nuteez localy lockal local gas lcal loal locl local llocal local. local farmer local plastic local, localp"
...,...,...
14186,[0],0
14187,[#],#
14188,[@],@
14189,[*],*


In [18]:
df_unique_match[:20]

,match,similar_strings
0,"[shalina healthcare, shelina healthcare, shalina health]",shalina healthcare ltd shalina healthcare kenya limited shalina healthcare kenya ltd shelina healthcare kenya limited shalina health kenya limited shalina health kenya
1,"[pi industries, pi industies, pt industries, p1 industries, ip industries]",pi industries ltd pt industries ltd p1 industries ltd ip industries ltd pi industies ltd
2,[kitale],kitale industries kitale indusries kitale industrues kitale indistries kitale industriez kitale induatries kitale millers kitale ltd kitale feeds kitale industry kitale kitale indust. kitale ltd kitale best kitale agritech kitale agriteeh kitale agreetec
3,"[avivet agro, avvivet agro, avivet ago, avivet. agro.]",avivet agro solution ltd avivet agro ltd avivet agro solutions lmd avivet agro sutions ltd avivet agro solutions limited avivet agro chemicals avivet. agro. solutions avivet agro solution limited avivet agro solutions avivet agro solution avivet agro soln ltd avivet ago solutions avvivet agro solution avivet agro sokutions avivet agro solution( avivet agro sol
4,"[local, lockal, localy, locals, localp, locall, local., local,, llocal, locl, loca, loal, lcal]","local manufacturer local juakali local manufacturers local manufaturer local manufacture local suppliers local seller local industries local ltd local ltd. local dealers locals local carpenters local any local potter localy made local bomet locall local supplier local indurtries local market loca local providers local nairobi local fabricators local products local artisan local welder local nuteez localy lockal local gas lcal loal locl local llocal local. local farmer local plastic local, localp"
5,"[sigma, sigima]",sigma limited sigma sigma company sigma ltd sigima sigma ltds sigma co. sigma feeds sigma feed
6,[bestvet agrosolution],bestvet agrosolution ltd
7,"[crown, crowns]",crown chemicals crown labs crown solution crown health crown healthcare crown limited crown healthcareltd crowns crown feeds crown paints crown ltd crown seeds crown bird crown heathcare crown crown paint
8,"[pl lndustries, balm lndustries]",pl lndustries ltd balm lndustries ltds
9,"[africasia seeds, africascia seeds, africasia seed, aficasia seed, african seeds]",africasia seeds company ltd africasia seed company ltda aficasia seed company ltd africasia seed co ltd africasia seed company limited africasia seed company limitedf africascia seeds ltd african seeds kenya africasia seeds kenya ltd africasia seeds k ltd africasia seed ltd africasia seed company africasia seed co.ltd africasia seed k ltd africasia seed kenya africasia seed co. africasia seeds co. africasia seed kenya ltd


### Topic Modeling

In [19]:
# extracting most common words from each cluster in order
cluster_word_freq = {}


for id, row in df_unique_match.iterrows():
    cluster = row['similar_strings']
    
    words = re.split(r'\s+|-|\(|\)|/|\\|\||,', cluster)
    for word in words:
        if id in cluster_word_freq:
            cluster_word_freq[id][word] = cluster_word_freq[id].get(word, 0) + 1
        else:
            cluster_word_freq[id] = {word: 1}
    
for id in cluster_word_freq:
    cluster_word_freq[id] = sorted(cluster_word_freq[id].items(), key=lambda x: x[1], reverse=True)
    # cluster_word_freq[id] = list(cluster_word_freq[id].items())
    # cluster_word_freq[id] = cluster_word_freq[id].items()

cluster_word_freq_df = pd.DataFrame.from_dict(cluster_word_freq.items())
cluster_word_freq_df.rename(columns={0: 'id', 1: 'word_freq'}, inplace=True)
cluster_word_freq_df.head()

,id,word_freq
0,0,"[(shalina, 5), (kenya, 5), (healthcare, 4), (limited, 3), (ltd, 2), (health, 2), (shelina, 1)]"
1,1,"[(ltd, 5), (industries, 4), (pi, 2), (pt, 1), (p1, 1), (ip, 1), (industies, 1)]"
2,2,"[(kitale, 17), (ltd, 2), (industries, 1), (indusries, 1), (industrues, 1), (indistries, 1), (industriez, 1), (induatries, 1), (millers, 1), (feeds, 1), (industry, 1), (indust., 1), (best, 1), (agritech, 1), (agriteeh, 1), (agreetec, 1)]"
3,3,"[(avivet, 14), (agro, 14), (solution, 5), (solutions, 5), (ltd, 4), (limited, 2), (lmd, 1), (sutions, 1), (chemicals, 1), (avivet., 1), (agro., 1), (soln, 1), (ago, 1), (avvivet, 1), (sokutions, 1), (, 1), (sol, 1)]"
4,4,"[(local, 30), (localy, 2), (manufacturer, 1), (juakali, 1), (manufacturers, 1), (manufaturer, 1), (manufacture, 1), (suppliers, 1), (seller, 1), (industries, 1), (ltd, 1), (ltd., 1), (dealers, 1), (locals, 1), (carpenters, 1), (any, 1), (potter, 1), (made, 1), (bomet, 1), (locall, 1), (supplier, 1), (indurtries, 1), (market, 1), (loca, 1), (providers, 1), (nairobi, 1), (fabricators, 1), (products, 1), (artisan, 1), (welder, 1), (nuteez, 1), (lockal, 1), (gas, 1), (lcal, 1), (loal, 1), (locl, 1), (llocal, 1), (local., 1), (farmer, 1), (plastic, 1), (, 1), (localp, 1)]"


In [109]:
cluster_word_freq_df['cluster_name'] = cluster_word_freq_df['word_freq'].apply(lambda x: ' '.join(word[0] for word in x[:2]))
cluster_word_freq_df.head()

,id,word_freq,cluster_name
0,0,"[(shalina, 5), (kenya, 5), (healthcare, 4), (limited, 3), (ltd, 2), (health, 2), (shelina, 1)]",shalina kenya
1,1,"[(ltd, 5), (industries, 4), (pi, 2), (pt, 1), (p1, 1), (ip, 1), (industies, 1)]",ltd industries
2,2,"[(kitale, 17), (ltd, 2), (industries, 1), (indusries, 1), (industrues, 1), (indistries, 1), (industriez, 1), (induatries, 1), (millers, 1), (feeds, 1), (industry, 1), (indust., 1), (best, 1), (agritech, 1), (agriteeh, 1), (agreetec, 1)]",kitale ltd
3,3,"[(avivet, 14), (agro, 14), (solution, 5), (solutions, 5), (ltd, 4), (limited, 2), (lmd, 1), (sutions, 1), (chemicals, 1), (avivet., 1), (agro., 1), (soln, 1), (ago, 1), (avvivet, 1), (sokutions, 1), (, 1), (sol, 1)]",avivet agro
4,4,"[(local, 30), (localy, 2), (manufacturer, 1), (juakali, 1), (manufacturers, 1), (manufaturer, 1), (manufacture, 1), (suppliers, 1), (seller, 1), (industries, 1), (ltd, 1), (ltd., 1), (dealers, 1), (locals, 1), (carpenters, 1), (any, 1), (potter, 1), (made, 1), (bomet, 1), (locall, 1), (supplier, 1), (indurtries, 1), (market, 1), (loca, 1), (providers, 1), (nairobi, 1), (fabricators, 1), (products, 1), (artisan, 1), (welder, 1), (nuteez, 1), (lockal, 1), (gas, 1), (lcal, 1), (loal, 1), (locl, 1), (llocal, 1), (local., 1), (farmer, 1), (plastic, 1), (, 1), (localp, 1)]",local localy


In [110]:
cluster_word_freq_df[:50]

,id,word_freq,cluster_name
0,0,"[(shalina, 5), (kenya, 5), (healthcare, 4), (limited, 3), (ltd, 2), (health, 2), (shelina, 1)]",shalina kenya
1,1,"[(ltd, 5), (industries, 4), (pi, 2), (pt, 1), (p1, 1), (ip, 1), (industies, 1)]",ltd industries
2,2,"[(kitale, 17), (ltd, 2), (industries, 1), (indusries, 1), (industrues, 1), (indistries, 1), (industriez, 1), (induatries, 1), (millers, 1), (feeds, 1), (industry, 1), (indust., 1), (best, 1), (agritech, 1), (agriteeh, 1), (agreetec, 1)]",kitale ltd
3,3,"[(avivet, 14), (agro, 14), (solution, 5), (solutions, 5), (ltd, 4), (limited, 2), (lmd, 1), (sutions, 1), (chemicals, 1), (avivet., 1), (agro., 1), (soln, 1), (ago, 1), (avvivet, 1), (sokutions, 1), (, 1), (sol, 1)]",avivet agro
4,4,"[(local, 30), (localy, 2), (manufacturer, 1), (juakali, 1), (manufacturers, 1), (manufaturer, 1), (manufacture, 1), (suppliers, 1), (seller, 1), (industries, 1), (ltd, 1), (ltd., 1), (dealers, 1), (locals, 1), (carpenters, 1), (any, 1), (potter, 1), (made, 1), (bomet, 1), (locall, 1), (supplier, 1), (indurtries, 1), (market, 1), (loca, 1), (providers, 1), (nairobi, 1), (fabricators, 1), (products, 1), (artisan, 1), (welder, 1), (nuteez, 1), (lockal, 1), (gas, 1), (lcal, 1), (loal, 1), (locl, 1), (llocal, 1), (local., 1), (farmer, 1), (plastic, 1), (, 1), (localp, 1)]",local localy
5,5,"[(sigma, 8), (limited, 1), (company, 1), (ltd, 1), (sigima, 1), (ltds, 1), (co., 1), (feeds, 1), (feed, 1)]",sigma limited
6,6,"[(bestvet, 1), (agrosolution, 1), (ltd, 1)]",bestvet agrosolution
7,7,"[(crown, 15), (chemicals, 1), (labs, 1), (solution, 1), (health, 1), (healthcare, 1), (limited, 1), (healthcareltd, 1), (crowns, 1), (feeds, 1), (paints, 1), (ltd, 1), (seeds, 1), (bird, 1), (heathcare, 1), (paint, 1)]",crown chemicals
8,8,"[(lndustries, 2), (pl, 1), (ltd, 1), (balm, 1), (ltds, 1)]",lndustries pl
9,9,"[(africasia, 15), (seed, 12), (ltd, 9), (seeds, 6), (company, 6), (kenya, 4), (k, 2), (co., 2), (ltda, 1), (aficasia, 1), (co, 1), (limited, 1), (limitedf, 1), (africascia, 1), (african, 1), (co.ltd, 1)]",africasia seed
